In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None

# Exercise 2.2: Mesh plots (psyplot)
prepared by A. Lauber

Here we learn how to plot data from an unstructured grid like the [ICON](https://code.mpimet.mpg.de/projects/iconpublic) grid as mesh grid.
There is the option to interpolate the data to a regular grid or to use the library [psyplot](https://psyplot.github.io), which was developed for plotting data on the unstructured grid.

Advantages of plotting on the unstructured grid:
- Shows the real output without interpolation
- No preprocessing of the data necessary

Disadvantages of plotting on the unstructured grid:
- It can be quite slow depending on the size of your netCDF file
- Not all features are available yet (psyplot is still under development)


## Import libraries

In [ ]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc
import urllib.request

## Download data

We will use a netCDF file created with an ICON run by Nadja Omanovic.
It was reduced to the 2m temperature over Switzerland to save space. The file is saved on the FTP server from IAC.

In [ ]:
ftp_pyvis = "ftp://iacftp.ethz.ch/pub_read/alauber/pyvis/"
filename = "my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc"
urllib.request.urlretrieve(ftp_pyvis + filename, "../data/" + filename)
filename = "rmp_my_exp1_atm_3d_ml_20180921T000000Z.nc"
urllib.request.urlretrieve(ftp_pyvis + filename, "../data/" + filename)

## Now have a look into the data

In [ ]:
ds = xr.open_dataset("../data/my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc")
ds

## Plot data with pcolormesh
Why does it not work?

In [ ]:
# Get data
lon, lat, temp = ds.clon, ds.clat, ds.t_2m[:, :, :]

# Code here

Answer: pcolormesh cannot handle data on an unstructered grid. For plotting the data with pcolormesh, the data has to be remapped to a structured grid first.

## Plot data with pcolormesh using remapped data

Have a look into the remapped data first:

In [ ]:
ds_rmp = xr.open_dataset("../data/rmp_my_exp1_atm_3d_ml_20180921T000000Z.nc")
ds_rmp

### Exercise
 * Plot the 2m temperature over Switzerland
 * Use `projection=ccrs.Robinson()` (don't forget to transform the data)
 * Add the borders of Switzerland
 * Cut off data outside of Switzerland (5.8<lon<10.7, 45.5<lat<48)
 * Add a colorbar
 * Set limits to the colorbar
 * Use the colormap `cmc.nuuk`

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0, 0, :, :]

# Code for plotting here

## Solution

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0, 0, :, :]

ax = plt.axes(projection=ccrs.Robinson())

h = ax.pcolormesh(
    lon, lat, temp, transform=ccrs.PlateCarree(), 
    cmap=cmc.nuuk, vmin=260, vmax=300)

ax.set_title("2m temperature in Switzerland")
ax.add_feature(cartopy.feature.BORDERS)

ax.set_extent([5.8, 10.7, 45.5, 48])
plt.colorbar(h)
plt.show()

## Let's do the same now on the original grid
We will now get started with [psyplot](https://psyplot.github.io). You can have a look at the website.

First we need to import the psyplot library:

In [ ]:
import psyplot.project as psy

# The following is needed to show plots after they are being updated
%config InlineBackend.close_figures = False
psy.rcParams["auto_show"] = True

Let's now take the file with the original ICON grid and plot it.

## Load data with psyplot

In [ ]:
# Load dataset with psyplot
ds_icon = psy.open_dataset("../data/my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc")
ds_icon

## Plot data with psyplot

In [ ]:
plot_icon = ds_icon.psy.plot.mapplot(name="t_2m")

I guess we could do that nicer....

Checkout the [available formatoptions](https://psyplot.github.io/psy-maps/generated/psyplot.project.plot.mapplot.html) and try to make the same plot as before.


### Exercise
 Use `plot_icon.update(...)` to adapt the plot in the following way: 
 * Plot the 2m temperature over Switzerland
 * Use `projection=ccrs.Robinson()` (don't forget to transform the data)
 * Cut off data outside of Switzerland (5.8<lon<10.7, 45.5<lat<48)
 * Set limits to the colorbar
 * Use the colormap `cmc.nuuk`
 
 (Hints: check map_extent; use bounds={'method':'..','vmin':'..'})

In [ ]:
plot_icon.update()  # Code here

### Solution

In [ ]:
plot_icon.update(
    projection=ccrs.Robinson(),
    map_extent=[5.8, 10.7, 45.5, 48],
    cmap=cmc.nuuk,
    bounds={"method": "minmax", "vmin": 270, "vmax": 300},
    title="2m temperature in Switzerland"
)

### Exercise
 * Add borders to the plot
 
Hint: the matplotlib axes can be accessed with `ax = plot_icon.plotters[0].ax`. Borders can then be added the same way as we added the lakes in exercise [ex2_0_intro_scatter.ipynb](ex2_0_intro_scatter.ipynb).

In [ ]:
# Add borders (don't forget to update the plot to see the plot)
import cartopy.feature as cf

# Code here

### Solution

In [ ]:
# Add borders (don't forget to update the plot to see the plot)
import cartopy.feature as cf

ax = plot_icon.plotters[0].ax
ax.add_feature(cf.BORDERS, edgecolor="0.1", zorder=100)
plot_icon.update()

Close the figure before creating a new one:

In [ ]:
plot_icon.close()

### Exercise formatoptions

Adding borders is still a bit complicated. A nicer way to add them is to use formatoptions, which you can generate yourself. The advantage is that they can be reused across different scripts. 

MeteoSwiss and C2SM developed [iconarray](https://github.com/C2SM/iconarray), which includes some formatoptions already like adding borders. Check out their [formatoptions](https://github.com/C2SM/iconarray#formatoptions) and repeat the exercise by using them.

In [ ]:
import iconarray

plot_icon = ds_icon.psy.plot.mapplot(...)

## Solution

In [ ]:
import iconarray

plot_icon = ds_icon.psy.plot.mapplot(
    name="t_2m",
    projection=ccrs.Robinson(),
    map_extent=[5.8, 10.7, 45.5, 48],
    cmap=cmc.nuuk,
    bounds={"method": "minmax", "vmin": 270, "vmax": 300},
)

Note that [iconarray](https://github.com/C2SM/iconarray) shows borders and lakes by default when being imported. If you don't want that, you need to actively turn them off:

In [ ]:
plot_icon.update(lakes=False)